In [7]:
import cv2
import matplotlib.pyplot as plt
import numpy as np


def getContours(img):
    """Get contour of template or sample image"""
    # according to opencv documentation, binary input is preferred, which can be get by applying threshold or 
    # canny edge detection 

    # thresholding method
    # Convert original image to gray scale
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Convert gray scale to binary
    threshold, img_binary = cv2.threshold(imgray, 80, 255, cv2.THRESH_BINARY)
    # Words are white, background is black, easy for findContour function
    img_binary_inv = cv2.bitwise_not(img_binary)
    # Dilation make all 6 to form a closed loop
    # kernel = np.ones((1, 1), np.uint8)
    # img_dilation = cv2.dilate(img_binary_inv, kernel, iterations=2)
    # Get Contour
    contours, hierarchy = cv2.findContours(img_binary_inv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    return contours  # [tmp for tmp in contours if tmp.shape[0] >= 100]


def getTemplateCV():
    """Get complex vector of template contour"""
    # Template region
    template_region = imgOricpy[rect[1]:rect[1] + rect[3], rect[0]:rect[0] + rect[2]]
    # Automatically find template contour
    tp_contour = getContours(template_region)

    for contour in tp_contour:
        x, y, w, h = cv2.boundingRect(contour)  # rectangle area bounding contour
        for point in contour:
            # -x and -y are to make left and upper boundry start from 0
            templateComVector.append(complex(point[0][0] - x, (point[0][1] - y)))
    return tp_contour


def getSampleCV():
    """Get complex vectors of all contours on sample image"""
    sp_contours = getContours(imgOricpy)

    for contour in sp_contours:
        sample_cv = []
        x, y, w, h = cv2.boundingRect(contour)  # Rectangle area bounding contour
        cv2.rectangle(imgOri, (x, y), (x + w, y + h), (100, 100, 100), 1)

        for point in contour:
            sample_cv.append(complex(point[0][0] - x, (point[0][1] - y)))
        # sampleComVectors store complex vectors of all sample contours
        sampleComVectors.append(sample_cv)
        # sampleContours store all sample contours, same order with sampleComVectors
        sampleContours.append(contour)
    return sp_contours


def getTemplateFD():
    """Apply fourier transform on template complex vectors to get fourier descriptor"""
    return np.fft.fft(templateComVector)


def getSampleFDs():
    """Apply fourier transform on all sample complex vectors to get fourier descriptor"""
    FDs = []
    for sample_cv in sampleComVectors:
        sampleFD = np.fft.fft(sample_cv)
        FDs.append(sampleFD)

    return FDs


def rotationInvariant(fourierDesc):
    """Make fourier descriptor invariant to rotation and start point"""
    for index, value in enumerate(fourierDesc):
        fourierDesc[index] = np.absolute(value)  # Consider only absolute value

    return fourierDesc


def scaleInvariant(fourierDesc):
    """Make fourier descriptor invariant to scale"""
    firstVal = fourierDesc[0]

    for index, value in enumerate(fourierDesc):
        fourierDesc[index] = value / firstVal  # Divided by first value

    return fourierDesc


def transInvariant(fourierDesc):
    """Make fourier descriptor invariant to translation"""
    return fourierDesc[1:]  # Drop the first coefficient


def getLowFreqFDs(fourierDesc):
    """Get the lowest X of frequency values from the fourier values."""
    # frequency order returned by np.fft is (0, 0.1, 0.2, 0.3, ...... , -0.3, -0.2, -0.1)
    # In transInvariant(), already remove first FD(0 frequency)

    return fourierDesc[:5]


In [8]:
print("Shape Matching Using Fourier Descriptor")

imgOri = cv2.imread(r"a4.bmp", 1)
imgOricpy = imgOri.copy()
img_2 = cv2.imread("sample_2.jpg")
img_c = cv2.imread("sample_c.jpg")

distThreshold = 0.06
rect = (64, 195, 171, 232)  # Rectangle area for 2
# rect = (489, 189, 134, 193)  # Rectangle area for C
templateComVector = []
sampleComVectors = []
sampleContours = []

# Get complex vector
template_contour = getTemplateCV()
sample_contour = getSampleCV()
# Get fourier descriptor
template_FD = getTemplateFD()
sampleFDs = getSampleFDs()

Shape Matching Using Fourier Descriptor


In [9]:
template_contour[0].shape

(1, 1, 2)

In [10]:
for i in range(len(sampleContours)):
    print(i, sampleContours[i].shape)

0 (1, 1, 2)
1 (22, 1, 2)
2 (2, 1, 2)
3 (1, 1, 2)
4 (1, 1, 2)
5 (2, 1, 2)
6 (1, 1, 2)
7 (2, 1, 2)
8 (1, 1, 2)
9 (9, 1, 2)
10 (1, 1, 2)
11 (1, 1, 2)
12 (1, 1, 2)
13 (2, 1, 2)
14 (1, 1, 2)
15 (29, 1, 2)
16 (48, 1, 2)
17 (1, 1, 2)
18 (1, 1, 2)
19 (1, 1, 2)
20 (2, 1, 2)
21 (4, 1, 2)
22 (2, 1, 2)
23 (1, 1, 2)
24 (1, 1, 2)
25 (13, 1, 2)
26 (1, 1, 2)
27 (1, 1, 2)
28 (1, 1, 2)
29 (1, 1, 2)
30 (21, 1, 2)
31 (1, 1, 2)
32 (8, 1, 2)
33 (1, 1, 2)
34 (1, 1, 2)
35 (13, 1, 2)
36 (1, 1, 2)
37 (1, 1, 2)
38 (1, 1, 2)
39 (4, 1, 2)
40 (1, 1, 2)
41 (1, 1, 2)
42 (2, 1, 2)
43 (1, 1, 2)
44 (1, 1, 2)
45 (1, 1, 2)
46 (63, 1, 2)
47 (4, 1, 2)
48 (1, 1, 2)
49 (1, 1, 2)
50 (4, 1, 2)
51 (6, 1, 2)
52 (1, 1, 2)
53 (6, 1, 2)
54 (8, 1, 2)
55 (1, 1, 2)
56 (1, 1, 2)
57 (1, 1, 2)
58 (3, 1, 2)
59 (2, 1, 2)
60 (1, 1, 2)
61 (1, 1, 2)
62 (2, 1, 2)
63 (4, 1, 2)
64 (1, 1, 2)
65 (4, 1, 2)
66 (1, 1, 2)
67 (1, 1, 2)
68 (1, 1, 2)
69 (1, 1, 2)
70 (2, 1, 2)
71 (2, 1, 2)
72 (1, 1, 2)
73 (1, 1, 2)
74 (2, 1, 2)
75 (1, 1, 2)
76 (3, 1, 2)
77

875 (6, 1, 2)
876 (1, 1, 2)
877 (3, 1, 2)
878 (1, 1, 2)
879 (4, 1, 2)
880 (1, 1, 2)
881 (1, 1, 2)
882 (1, 1, 2)
883 (38, 1, 2)
884 (1, 1, 2)
885 (2, 1, 2)
886 (17, 1, 2)
887 (1, 1, 2)
888 (1, 1, 2)
889 (2, 1, 2)
890 (3, 1, 2)
891 (1, 1, 2)
892 (2, 1, 2)
893 (6, 1, 2)
894 (1, 1, 2)
895 (1, 1, 2)
896 (1, 1, 2)
897 (2, 1, 2)
898 (1, 1, 2)
899 (2, 1, 2)
900 (1, 1, 2)
901 (1, 1, 2)
902 (4, 1, 2)
903 (2, 1, 2)
904 (41, 1, 2)
905 (1, 1, 2)
906 (1, 1, 2)
907 (1, 1, 2)
908 (1, 1, 2)
909 (11, 1, 2)
910 (1, 1, 2)
911 (2, 1, 2)
912 (4, 1, 2)
913 (2, 1, 2)
914 (6, 1, 2)
915 (1, 1, 2)
916 (2, 1, 2)
917 (1, 1, 2)
918 (1, 1, 2)
919 (1, 1, 2)
920 (1, 1, 2)
921 (12, 1, 2)
922 (1, 1, 2)
923 (2, 1, 2)
924 (1, 1, 2)
925 (14, 1, 2)
926 (2, 1, 2)
927 (14, 1, 2)
928 (1, 1, 2)
929 (1, 1, 2)
930 (1, 1, 2)
931 (8, 1, 2)
932 (1, 1, 2)
933 (21, 1, 2)
934 (1, 1, 2)
935 (1, 1, 2)
936 (54, 1, 2)
937 (1, 1, 2)
938 (1, 1, 2)
939 (125, 1, 2)
940 (2, 1, 2)
941 (2, 1, 2)
942 (1, 1, 2)
943 (2, 1, 2)
944 (1, 1, 2)
945 (13, 

1733 (8, 1, 2)
1734 (3, 1, 2)
1735 (1, 1, 2)
1736 (1, 1, 2)
1737 (1, 1, 2)
1738 (1, 1, 2)
1739 (1, 1, 2)
1740 (1, 1, 2)
1741 (1, 1, 2)
1742 (3, 1, 2)
1743 (1, 1, 2)
1744 (2, 1, 2)
1745 (1, 1, 2)
1746 (1, 1, 2)
1747 (1, 1, 2)
1748 (1, 1, 2)
1749 (1, 1, 2)
1750 (1, 1, 2)
1751 (1, 1, 2)
1752 (2, 1, 2)
1753 (1, 1, 2)
1754 (1, 1, 2)
1755 (1, 1, 2)
1756 (1, 1, 2)
1757 (2, 1, 2)
1758 (2, 1, 2)
1759 (1, 1, 2)
1760 (3, 1, 2)
1761 (1, 1, 2)
1762 (1, 1, 2)
1763 (1, 1, 2)
1764 (1, 1, 2)
1765 (1, 1, 2)
1766 (1, 1, 2)
1767 (1, 1, 2)
1768 (31, 1, 2)
1769 (2, 1, 2)
1770 (1, 1, 2)
1771 (1, 1, 2)
1772 (1, 1, 2)
1773 (1, 1, 2)
1774 (1, 1, 2)
1775 (1, 1, 2)
1776 (2, 1, 2)
1777 (1, 1, 2)
1778 (2, 1, 2)
1779 (1, 1, 2)
1780 (6, 1, 2)
1781 (1, 1, 2)
1782 (4, 1, 2)
1783 (1, 1, 2)
1784 (3, 1, 2)
1785 (1, 1, 2)
1786 (1, 1, 2)
1787 (1, 1, 2)
1788 (1, 1, 2)
1789 (1, 1, 2)
1790 (1, 1, 2)
1791 (1, 1, 2)
1792 (1, 1, 2)
1793 (272, 1, 2)
1794 (1, 1, 2)
1795 (2, 1, 2)
1796 (1, 1, 2)
1797 (2, 1, 2)
1798 (1, 1, 2)
1799 (1

In [11]:
cv2.drawContours(imgOri, sampleContours, -1, (0, 255, 0), 8)

array([[[241, 241, 241],
        [215, 215, 215],
        [161, 161, 161],
        ...,
        [248, 248, 248],
        [246, 246, 246],
        [239, 239, 239]],

       [[220, 220, 220],
        [229, 229, 229],
        [208, 208, 208],
        ...,
        [240, 240, 240],
        [191, 191, 191],
        [232, 232, 232]],

       [[164, 164, 164],
        [229, 229, 229],
        [229, 229, 229],
        ...,
        [247, 247, 247],
        [242, 242, 242],
        [244, 244, 244]],

       ...,

       [[222, 222, 222],
        [192, 192, 192],
        [141, 141, 141],
        ...,
        [242, 242, 242],
        [243, 243, 243],
        [246, 246, 246]],

       [[169, 169, 169],
        [187, 187, 187],
        [216, 216, 216],
        ...,
        [250, 250, 250],
        [250, 250, 250],
        [255, 255, 255]],

       [[131, 131, 131],
        [202, 202, 202],
        [221, 221, 221],
        ...,
        [243, 243, 243],
        [233, 233, 233],
        [241, 241, 241]]

In [12]:
cv2.imshow('img', imgOri)
k = cv2.waitKey(0)
if k == 27:         # wait for ESC key to exit
    cv2.destroyAllWindows()